# Teste Geracao Filtro

- Notebook para geracao dos coeficientes do filtro utilizado no projeto.
- Filtro utilizado:  
    - Filtro digital **IIR** _(Infinite Impulse Response)_ do tipo **Biquad Cascade**, utilizando as funções otimizadas da biblioteca CMSIS-DSP da ARM (no Microcontrolador).
    - Tipo **Chebyshev**
    - Frequencia de amostragem do sinal: **48 kHz**
    - Passa faixa:
        - Frequencia de corte inferior: **20 Hz**
        - Frequencia de corte superior: **20 kHz**


In [ ]:
import scipy.signal as signal
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from tg_utils import GeradorFiltroPassaFaixa, visualizar_graficos_do_filtro


In [ ]:
# Especificações do filtro baseadas no artigo
FS_FILTER = 48000      # Frequência de amostragem do filtro em Hz
LOWCUT = 20.0         # Frequência de corte inferior em Hz
HIGHCUT = 20000.0      # Frequência de corte superior em Hz
ORDER = 4              # Ordem do filtro
FILTER_TYPE = 'butter' # Opções: 'butter', 'cheby1', 'cheby2'
RP = 0.5               # Ripple máximo na banda de passagem em dB (apenas para Chebyshev)


In [ ]:
def generate_fir_filter_coeffs():
    """
    Projeta um filtro FIR passa-faixa e gera os coeficientes no formato C,
    além de visualizar a resposta do filtro.
    """
    # --- 1. Especificações do Filtro ---
    SAMPLE_RATE = 48000.0
    LOWCUT = 50.0      # Frequência de corte inferior em Hz (um pouco mais alta para estabilidade)
    HIGHCUT = 18000.0  # Frequência de corte superior em Hz (um pouco mais baixa para estabilidade)
    
    # NUM_TAPS é o parâmetro mais importante do filtro FIR.
    # É o número de coeficientes (a "ordem" do filtro).
    # - Maior NUM_TAPS: Filtro mais "afiado" e preciso, mas exige MAIS processamento.
    # - Menor NUM_TAPS: Filtro mais "suave", mas exige MENOS processamento.
    # Um número ímpar é geralmente preferido. Começaremos com 129, que é um valor razoável.
    NUM_TAPS = 129

    # --- 2. Geração dos Coeficientes (Taps) ---
    # Usamos a função firwin, ideal para filtros FIR baseados no método da janela.
    # pass_zero=False especifica que é um filtro passa-faixa (ou rejeita-faixa).
    taps = signal.firwin(NUM_TAPS, [LOWCUT, HIGHCUT], fs=SAMPLE_RATE, pass_zero=False)

    # --- 3. Geração do Código C ---
    print("--- Cole este código no seu main.c ---")
    print(f"// Coeficientes do Filtro FIR Passa-Faixa ({NUM_TAPS} taps)")
    print(f"// Fs={SAMPLE_RATE}Hz, Fc=[{LOWCUT}Hz, {HIGHCUT}Hz]")
    print(f"#define NUM_TAPS {NUM_TAPS}")
    print("const float32_t firCoeffs[NUM_TAPS] = {")
    for i, c in enumerate(taps):
        # Adiciona uma quebra de linha a cada 5 coeficientes para facilitar a leitura
        if i > 0 and i % 5 == 0:
            print("")
        print(f"    {c:.8f}f,", end='')
    print("\n};")
    print("-----------------------------------------\n")


    # --- 4. Visualização do Filtro ---
    # Calcula a resposta de frequência do filtro
    w, h = signal.freqz(taps, 1, worN=8000, fs=SAMPLE_RATE)
    db_h = 20 * np.log10(np.abs(h))

    # Modificado para 3 subplots para adicionar o gráfico de zoom
    fig, axs = plt.subplots(3, 1, figsize=(12, 15))
    fig.suptitle('Análise do Filtro FIR Gerado', fontsize=16)

    # Gráfico 1: Resposta ao Impulso (os próprios coeficientes)
    axs[0].plot(taps, '.-')
    axs[0].set_title(f'Resposta ao Impulso ({NUM_TAPS} Taps)')
    axs[0].set_xlabel('Índice do Coeficiente')
    axs[0].set_ylabel('Amplitude')
    axs[0].grid(True)

    # Gráfico 2: Resposta de Frequência (Completa)
    axs[1].plot(w, db_h)
    axs[1].set_title('Resposta de Frequência (Visão Completa)')
    axs[1].set_xlabel('Frequência (Hz)')
    axs[1].set_ylabel('Ganho (dB)')
    axs[1].axvline(LOWCUT, color='red', linestyle='--', alpha=0.7, label=f'Corte Inferior ({LOWCUT} Hz)')
    axs[1].axvline(HIGHCUT, color='red', linestyle='--', alpha=0.7, label=f'Corte Superior ({HIGHCUT} Hz)')
    axs[1].set_ylim(-100, 5)
    axs[1].set_xlim(0, SAMPLE_RATE / 2)
    axs[1].legend()
    axs[1].grid(True)
    
    # Gráfico 3: Resposta de Frequência (Zoom na Faixa de Corte Inferior)
    axs[2].plot(w, db_h)
    axs[2].set_title('Resposta de Frequência (Zoom na Faixa de Corte Inferior)')
    axs[2].set_xlabel('Frequência (Hz)')
    axs[2].set_ylabel('Ganho (dB)')
    axs[2].axvline(LOWCUT, color='red', linestyle='--', alpha=0.7, label=f'Corte Inferior ({LOWCUT} Hz)')
    axs[2].set_xlim(0, LOWCUT * 4) # Define o limite X para 4x a frequência de corte inferior
    axs[2].set_ylim(-100, 5)
    axs[2].legend()
    axs[2].grid(True, which='both')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [ ]:
if __name__ == '__main__':
    # Peça ao usuário para fornecer o caminho do arquivo .wav
    # Use áudios gravados pelo seu setup para ter uma representação fiel.
    # Exemplo de caminho: "C:/Users/SeuNome/Documentos/TCC/audios/furadeira_saudavel.wav"
    # Ou simplesmente: "furadeira_saudavel.wav" se estiver na mesma pasta do script.
    
    print("Este script carrega um arquivo .wav, aplica um filtro e exibe os resultados.")
    print(f"Tipo de filtro configurado: {FILTER_TYPE.capitalize()}")
    # file_path = input("\nPor favor, insira o caminho para o arquivo .wav (com amostragem de 48000 Hz): ")
    file_path = "/home/dev/Workspace/TG/projeto-tg/python_code/data/audio_files/off.wav"

    g_filtro = GeradorFiltroPassaFaixa(lowcut=LOWCUT, highcut=HIGHCUT, fs=FS_FILTER, order=ORDER, filter_type=FILTER_TYPE, ripple=RP)
    filtro = g_filtro.projetar_filtro()

    visualizar_graficos_do_filtro(filtro, file_path)
    # generate_fir_filter_coeffs()